In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import math
import sys
import random
import pickle
import csv

import dask.dataframe as dd
from dask.distributed import Client

import torch
import pyro
import pyro.distributions as dist
import pyro.distributions.constraints as constraints
from pyro.nn import PyroModule
from pyro.infer import Predictive

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '/home/djl34/lab_pd/kl/git/KL/raklette')
import raklette_updated
from run_raklette import run_raklette
from run_raklette import TSVDataset

##############################################################################################################

KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch3/users/d/djl34"

base_set = ["A", "C", "T", "G"]
chrom_set = [str(x) for x in range(1, 23)]
# chrom_set = ["22"]

/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load files

In [28]:
variants = "1_split_500_window1k_570.tsv"
input_filename = variants

# length_file = os.path.join(scratch_dir, "kl_input/{header}_length.txt")
# length_file = length_file.replace("{header}", header)
neutral_sfs = KL_data_dir + "/whole_genome/neutral/5_bins/all.tsv"
n_covs = 0

df = pd.read_csv(variants, sep = "\t")
nb_samples = len(df)
n_genes = 1

print("number of samples: " + str(nb_samples))
print("number of genes: " + str(n_genes))

   
with open(variants) as f:
    first_line = f.readline()
header = first_line.split("\t")
header = [x.strip() for x in header]

chunksize = 1000000

print("number of chunks " + str(nb_samples/chunksize))

dataset = TSVDataset(input_filename, chunksize=chunksize, nb_samples = nb_samples, header_all = header, features = header)
loader = DataLoader(dataset, batch_size=1, num_workers=1, shuffle=False)

num_epochs = 2000

number of samples: 2955
number of genes: 1
number of chunks 0.002955


In [37]:
pyro.clear_param_store()


In [38]:
guide

AutoNormal(
  (locs): PyroModule()
  (scales): PyroModule()
)

In [35]:
pyro.get_param_store().load("1_split_500_window1k_570.pkl",)

In [39]:
pyro.param

<function pyro.primitives.param(name, init_tensor=None, constraint=Real(), event_dim=None)>

In [52]:
with open('1_split_500_window1k_570.model', 'rb') as handle:
    dictionary = pickle.load(handle)

In [53]:
dictionary

{'model': <bound method raklette.model of <raklette_updated.raklette object at 0x7f363557fa60>>,
 'guide': AutoNormal(
   (locs): PyroModule()
   (scales): PyroModule()
 )}

In [42]:
guide

AutoNormal(
  (locs): PyroModule()
  (scales): PyroModule()
)

In [51]:
guide.quantiles([0.05, 0.5, 0.95])

{'beta_sel': tensor([[[-4.7887,  2.2967, -0.6386,  0.2012]],
 
         [[-4.6885,  2.4627, -0.3943,  0.5629]],
 
         [[-4.5883,  2.6288, -0.1501,  0.9245]]])}

In [43]:
predictive = Predictive(model, guide=guide, num_samples = 800)

In [44]:
mu_bin = 2

df_mu = df[df["mu_index"] == mu_bin]

In [45]:
x = df_mu[header]
x = x.astype(float)
x = torch.from_numpy(x.values)

In [46]:
gene_col = 2
mu_col = 0
bin_col = 1

gene_ids = x[:,gene_col].reshape(-1)
gene_ids = gene_ids.type(torch.LongTensor)

mu_vals = x[:,mu_col].reshape(-1)
mu_vals = mu_vals.type(torch.LongTensor)

freq_bins = x[:,bin_col].reshape(-1)

In [47]:
samples = predictive(mu_vals, gene_ids, None)

In [48]:
df_mu_sfs = pd.DataFrame((df_mu.groupby("Freq_bin").size())/len(df_mu)).reset_index()

for i in range(5):
    if len(df_mu_sfs[df_mu_sfs["Freq_bin"] == i]) == 0:
        df_mu_sfs.loc[i] = [i, 0.0]
    
df_mu_sfs.rename({0: "observed"}, axis = 1, inplace = True)

# df_mu_sfs["neutral"] = pd.Series(df_neutral.iloc[mu_bin, 1:6]).reset_index(drop = True)


In [49]:
df_post_prediction = pd.DataFrame(samples["obs"].reshape(-1))
df_mu_sfs["post pred"] = list(pd.Series(df_post_prediction.groupby(0).size()/len(df_post_prediction)))

In [50]:
df_mu_sfs

,Freq_bin,observed,post pred
0,0.0,0.921053,0.879235
1,1.0,0.069079,0.107850
2,2.0,0.003289,0.009605
3,3.0,0.006579,0.003080
4,4.0,0.000000,0.000230
